In [9]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random
%matplotlib inline



In [8]:
import sys
sys.path
sys.path.insert(0, '/Users/asaadeldin/Downloads/GitHub/graspy')

In [7]:
from graspy.match import SeededGraphMatching as SGM

In [4]:
def SGMtest(A,B,pi):
    n = A.shape[0]
    for m in range(1,5):
        ofvs = [0]*30
        for i in range(30):
            W1 = np.sort(random.sample(list(range(n)),m))
            W2 = np.array([pi[z] for z in W1])
            faq = SGM(shuffle_input=False)
            faq = faq.fit(A,B,W1,W2)
            ofvs[i] = faq.score_
        print("OFV with " + str(m) + " seeds: "+ str(np.mean(ofvs)))

In [10]:
with open("qapdata/chr12c.dat") as f:
    f = [int(elem) for elem in f.read().split()]
# adjusting
f = np.array(f[1:])
n = int(math.sqrt(len(f) / 2))
f = f.reshape(2 * n, n)
A = f[:n, :]
B = f[n:, :]
pi = np.array([7,5,1,3,10,4,8,6,9,11,2,12])-[1]*n
W1 = np.sort(random.sample(list(range(n)),3))
W2 = np.array([pi[z] for z in W1])
#faq = SGM(n_init=100,init_method="rand",shuffle_input=False)
faq = SGM()
faq = faq.fit(A,B,W1,W2)
print(W1)
print(W2)
print(pi)
print(faq.perm_inds_)
print(faq.score_)
print(np.trace(np.transpose(A) @ B[np.ix_(faq.perm_inds_, faq.perm_inds_)]))


[ 3  7 10]
[2 5 1]
[ 6  4  0  2  9  3  7  5  8 10  1 11]
[ 6  4  0  2  9  3  7  5  8 10  1 11]
11156
11156


In [5]:
with open("qapdata/chr12c.dat") as f:
    f = [int(elem) for elem in f.read().split()]
# adjusting
f = np.array(f[1:])
n = int(math.sqrt(len(f) / 2))
f = f.reshape(2 * n, n)
A = f[:n, :]
B = f[n:, :]
pi = np.array([7,5,1,3,10,4,8,6,9,11,2,12])-[1]*n
SGMtest(A,B,pi)

OFV with 1 seeds: 19749.0
OFV with 2 seeds: 17739.6
OFV with 3 seeds: 15391.133333333333
OFV with 4 seeds: 14121.066666666668


In [60]:
with open('qapdata/chr15a.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])   
n = int(math.sqrt(len(f)/2))

f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]
pi = np.array([5,10,8,13,12,11,14,2,4,6,7,15,3,1,9])-[1]*n

SGMtest(A,B,pi)

OFV with 1 seeds: 15472.866666666667
OFV with 2 seeds: 16917.533333333333
OFV with 3 seeds: 14576.4
OFV with 4 seeds: 14623.066666666668


In [48]:
with open('qapdata/chr15c.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])   
n = int(math.sqrt(len(f)/2))

f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]
pi = np.array([13,2,5,7,8,1,14,6,4,3,15,9,12,11,10])-[1]*n

SGMtest(A,B,pi)

OFV with 1 seeds: 18502.4
OFV with 2 seeds: 15190.6
OFV with 3 seeds: 15402.933333333332
OFV with 4 seeds: 14349.0


In [49]:
with open('qapdata/chr20b.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])   
n = int(math.sqrt(len(f)/2))

f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]
pi = np.array([20,3,9,7,1,12,16,6,8,14,10,4,5,13,17,2,18,11,19,15])-[1]*n

SGMtest(A,B,pi)

OFV with 1 seeds: 3632.4666666666667
OFV with 2 seeds: 3519.2
OFV with 3 seeds: 3453.8
OFV with 4 seeds: 3134.0


In [50]:
with open('qapdata/chr22b.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])   
n = int(math.sqrt(len(f)/2))

f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]
pi = np.array([10,19,3,1,20,2,6,4,7,8,17,12,11,15,21,13,9,5,22,14,18,16])-[1]*n

SGMtest(A,B,pi)

OFV with 1 seeds: 8333.466666666667
OFV with 2 seeds: 8034.266666666666
OFV with 3 seeds: 7805.333333333333
OFV with 4 seeds: 7752.6


In [51]:
with open('qapdata/esc16b.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])   
n = int(math.sqrt(len(f)/2))

f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]
pi = np.array([6,3,7,5,13,1,15,2,4,11,9,14,10,12,8,16])-[1]*n

SGMtest(A,B,pi)

OFV with 1 seeds: 294.73333333333335
OFV with 2 seeds: 294.06666666666666
OFV with 3 seeds: 293.2
OFV with 4 seeds: 293.73333333333335


In [52]:
with open('qapdata/rou12.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])   
n = int(math.sqrt(len(f)/2))

f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]
pi = np.array([6,5,11,9,2,8,3,1,12,7,4,10])-[1]*n

SGMtest(A,B,pi)

OFV with 1 seeds: 250666.86666666667
OFV with 2 seeds: 241117.6
OFV with 3 seeds: 238694.0
OFV with 4 seeds: 236109.06666666668


In [53]:
with open('qapdata/rou15.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])   
n = int(math.sqrt(len(f)/2))

f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]
pi = np.array([12,6,8,13,5,3,15,2,7,1,9,10,4,14,11])-[1]*n

SGMtest(A,B,pi)

OFV with 1 seeds: 368860.2
OFV with 2 seeds: 358697.06666666665
OFV with 3 seeds: 358257.86666666664
OFV with 4 seeds: 355367.4666666667


In [54]:
with open('qapdata/rou20.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])   
n = int(math.sqrt(len(f)/2))

f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]
pi = np.array([1,19,2,14,10,16,11,20,9,5,7,4,8,18,15,3,12,17,13,6])-[1]*n

SGMtest(A,B,pi)

OFV with 1 seeds: 746158.4666666667
OFV with 2 seeds: 747792.3333333334
OFV with 3 seeds: 745508.0
OFV with 4 seeds: 735216.6666666666


In [55]:
with open('qapdata/tai15a.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])   
n = int(math.sqrt(len(f)/2))

f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]
pi = np.array([5,10,4,13,2,9,1,11,12,14,7,15,3,8,6])-[1]*n

SGMtest(A,B,pi)

OFV with 1 seeds: 400170.13333333336
OFV with 2 seeds: 402495.26666666666
OFV with 3 seeds: 399453.6666666667
OFV with 4 seeds: 394092.6666666667


In [56]:
with open('qapdata/tai17a.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])   
n = int(math.sqrt(len(f)/2))

f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]
pi = np.array([12,2,6,7,4,8,14,5,11,3,16,13,17,9,1,10,15])-[1]*n

SGMtest(A,B,pi)

OFV with 1 seeds: 515560.3333333333
OFV with 2 seeds: 510177.5333333333
OFV with 3 seeds: 510174.4666666667
OFV with 4 seeds: 501128.4


In [57]:
with open('qapdata/tai20a.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])   
n = int(math.sqrt(len(f)/2))

f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]
pi = np.array([10,9,12,20,19,3,14,6,17,11,5,7,15,16,18,2,4,8,13,1])-[1]*n

SGMtest(A,B,pi)

OFV with 1 seeds: 735769.3333333334
OFV with 2 seeds: 731969.0666666667
OFV with 3 seeds: 724333.6
OFV with 4 seeds: 720749.8


In [58]:
with open('qapdata/tai30a.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])   
n = int(math.sqrt(len(f)/2))

f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]
pi = np.array([19, 18, 4, 24, 30, 25, 5, 7, 1, 22, 28, 20, 11,13, 9, 16, 8, 10, 17, 21, 12, 29, 2, 15, 3, 14, 26, 27, 23, 6])-[1]*n

SGMtest(A,B,pi)

OFV with 1 seeds: 1883309.3333333333
OFV with 2 seeds: 1871592.1333333333
OFV with 3 seeds: 1867979.0
OFV with 4 seeds: 1859929.2666666666


In [59]:
with open('qapdata/tai35a.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])   
n = int(math.sqrt(len(f)/2))

f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]
pi = np.array([19,9,28,12,7,33,13,26,5,2,31,16,24,27,22,15,3,30,29,11,6,25,21,23,34,20,18,4,10,1,14,8,32,35,17])-[1]*n

SGMtest(A,B,pi)

OFV with 1 seeds: 2501301.7333333334
OFV with 2 seeds: 2502449.533333333
OFV with 3 seeds: 2499441.2666666666
OFV with 4 seeds: 2489536.8


In [10]:
import numpy as np
import math
from scipy.optimize import linear_sum_assignment
from scipy.optimize import minimize_scalar
from sklearn.utils import check_array
from sklearn.utils import column_or_1d
from graspy.match import SinkhornKnopp


class SGM1:
    """
    Seeded Graph Matching Algorithm (SGM)
    The seeded graph matching problem is a variation of graph matching
     in which part of the matching is fixed. This algorithm is a modification
     of FAQ, an algorithm also implemented in this package[1].


    Parameters
    ----------

    n_init : int, positive (default = 1)
        Number of random initializations of the starting permutation matrix that
        the FAQ algorithm will undergo. n_init automatically set to 1 if
        init_method = 'barycenter'

    init_method : string (default = 'barycenter')
        The initial position chosen

        "barycenter" : the non-informative “flat doubly stochastic matrix,”
        :math:`J=1*1^T /n` , i.e the barycenter of the feasible region

        "rand" : some random point near :math:`J, (J+K)/2`, where K is some random doubly
        stochastic matrix

    max_iter : int, positive (default = 30)
        Integer specifying the max number of FW iterations.
        FAQ typically converges with modest number of iterations

    shuffle_input : bool (default = True)
        Gives users the option to shuffle the nodes of A matrix to avoid results
        from inputs that were already matched

    eps : float (default = 0.1)
        A positive, threshold stopping criteria such that FW continues to iterate
        while Frobenius norm of :math:`(P_{i}-P_{i+1}) > eps`


    gmp : bool (default = False)
        Gives users the option to the Graph Matching Problem (GMP) rather than
        the Quadratic Assignment (QAP). This is accomplished through trivial
        negation of the objective function.

    Attributes
    ----------

    perm_inds_ : array, size (n,) where n is the number of vertices in the graphs fitted.
        The indices of the optimal permutation on the nodes of B, found via
        FAQ, to best minimize the objective function :math:`f(P) = trace(A^T PBP^T )`.


    score_ : float
        The objective function value of for the optimal permutation found.


    References
    ----------
    .. [1] J. T. Vogelstein, J. M. Conroy, V. Lyzinski, L. J. Podrazik, S. G. Kratzer,
           E. T. Harley, D. E. Fishkind, R. J. Vogelstein, and C. E. Priebe, “Fast
           approximate quadratic programming for graph matching,” PLOS one, vol. 10,
           no. 4, p. e0121002, 2015.



    """

    def __init__(
        self,
        n_init=1,
        init_method="barycenter",
        max_iter=30,
        shuffle_input=True,
        eps=0.1,
        gmp=False,
    ):

        if n_init > 0 and type(n_init) is int:
            self.n_init = n_init
        else:
            msg = '"n_init" must be a positive integer'
            raise TypeError(msg)
        if init_method == "rand":
            self.init_method = "rand"
        elif init_method == "barycenter":
            self.init_method = "barycenter"
            self.n_init = 1
        else:
            msg = 'Invalid "init_method" parameter string'
            raise ValueError(msg)
        if max_iter > 0 and type(max_iter) is int:
            self.max_iter = max_iter
        else:
            msg = '"max_iter" must be a positive integer'
            raise TypeError(msg)
        if type(shuffle_input) is bool:
            self.shuffle_input = shuffle_input
        else:
            msg = '"shuffle_input" must be a boolean'
            raise TypeError(msg)
        if eps > 0 and type(eps) is float:
            self.eps = eps
        else:
            msg = '"eps" must be a positive float'
            raise TypeError(msg)
        if type(gmp) is bool:
            self.gmp = gmp
        else:
            msg = '"gmp" must be a boolean'
            raise TypeError(msg)

    def fit(self, A, B, W1=[], W2=[]):
        """
        Fits the model with two assigned adjacency matrices

        Parameters
        ---------
        A : 2d-array, square, positive
            A square, positive adjacency matrix

        B : 2d-array, square, positive
            A square, positive adjacency matrix

        W1 : 1d-array, shape (m , 1) where m <= n
            An array where each entry is a node in A

        W2 : 1d-array, shape (m , 1) where m <= n
            An array where each entry is a node in B
            The elements of W1 and W2 are seeds, creating a fixed
            seeding of W1 -> W2

        Returns
        -------

        self : returns an instance of self
        """
        A = check_array(A, copy=True, ensure_2d=True)
        B = check_array(B, copy=True, ensure_2d=True)
        W1 = column_or_1d(W1)
        W2 = column_or_1d(W2)
        
        if A.shape[0] != B.shape[0]:
            msg = "Adjacency matrices must be of equal size"
            raise ValueError(msg)
        elif A.shape[0] != A.shape[1] or B.shape[0] != B.shape[1]:
            msg = "Adjacency matrix entries must be square"
            raise ValueError(msg)
        elif (A >= 0).all() == False or (B >= 0).all() == False:
            msg = "Adjacency matrix entries must be greater than or equal to zero"
            raise ValueError(msg)
        elif W1.shape[0] != W2.shape[0]:
            msg = "Seed arrays must be of equal size"
            raise ValueError(msg)
        elif W1.shape[0] > A.shape[0]:
            msg = "There cannot be more seeds than there are nodes"
            raise ValueError(msg)
        elif (W1 >= 0).all() == False or (W2 >= 0).all() == False:
            msg = "Seed array entries must be greater than or equal to zero"
            raise ValueError(msg)

        n = A.shape[0]  # number of vertices in graphs
        n_seeds = W1.shape[0]  # number of seeds
        n_unseed = n - n_seeds

        obj_func_scalar = 1
        if self.gmp:
            obj_func_scalar = -1
        
        if self.shuffle_input:
            W2_c = np.random.permutation(np.array([x for x in range(n) if x not in W2])
                                        )
        else:
            W2_c = np.array([x for x in range(n) if x not in W2])
            
        W1_c = np.array([x for x in range(n) if x not in W1])
        
        p_A = np.concatenate(
            [W1, W1_c], axis=None
        ).astype(int)
        p_B = np.concatenate(
            [W2, W2_c], axis=None
        ).astype(int)
        print(p_A)
        A = A[np.ix_(p_A, p_A)]
        B = B[np.ix_(p_B, p_B)]

        A11 = A[:n_seeds, :n_seeds]
        A12 = A[:n_seeds, n_seeds:]
        A21 = A[n_seeds:, :n_seeds]
        A22 = A[n_seeds:, n_seeds:]
        B11 = B[:n_seeds, :n_seeds]
        B12 = B[:n_seeds, n_seeds:]
        B21 = B[n_seeds:, :n_seeds]
        B22 = B[n_seeds:, n_seeds:]
        A11T = np.transpose(A11)
        A12T = np.transpose(A12)
        A22T = np.transpose(A22)
        B21T = np.transpose(B21)
        B22T = np.transpose(B22)

        score = math.inf
        perm_inds = np.zeros(n)

        for i in range(self.n_init):

            # setting initialization matrix
            if self.init_method == "rand":
                sk = SinkhornKnopp()
                K = np.random.rand(
                    n - n_seeds, n - n_seeds
                )  # generate a nxn matrix where each entry is a random integer [0,1]
                for i in range(10):  # perform 10 iterations of Sinkhorn balancing
                    K = sk.fit(K)
                J = np.ones((n - n_seeds, n - n_seeds)) / float(
                    n - n_seeds
                )  # initialize J, a doubly stochastic barycenter
                P = (K + J) / 2
            elif self.init_method == "barycenter":
                P = np.ones((n - n_seeds, n - n_seeds)) / float(n - n_seeds)

            const_sum = A21 @ np.transpose(B21) + np.transpose(A12) @ B12
            grad_P = math.inf  # gradient of P
            n_iter = 0  # number of FW iterations

            # OPTIMIZATION WHILE LOOP BEGINS
            while grad_P > self.eps and n_iter < self.max_iter:
                delta_f = (
                    const_sum + A22 @ P @ B22T + A22T @ P @ B22
                )  # computing the gradient of f(P) = -tr(APB^tP^t)
                rows, cols = linear_sum_assignment(
                    obj_func_scalar * delta_f
                )  # run hungarian algorithm on gradient(f(P))
                Q = np.zeros((n_unseed, n_unseed))
                Q[rows, cols] = 1  # initialize search direction matrix Q

                def f(x):  # computing the original optimization function
                    return obj_func_scalar * np.trace(A11T@B11) + np.trace(np.transpose(x*P+(1-x)*Q)@A21@B21T) + np.trace(np.transpose(x*P+(1-x)*Q)@A12T@B12) + np.trace(A22T@(x*P+(1-x)*Q)@B22@np.transpose(x*P+(1-x)*Q)) 

                alpha = minimize_scalar(
                    f, bounds=(0, 1), method="bounded"
                ).x  # computing the step size
                P_i1 = alpha * P + (1 - alpha) * Q  # Update P
                grad_P = np.linalg.norm(P - P_i1)
                P = P_i1
                n_iter += 1
            # end of FW optimization loop

            row, col = linear_sum_assignment(
                -P
            )  # Project onto the set of permutation matrices
            
            perm_inds_new = np.concatenate(
                (np.arange(n_seeds), np.array([x + n_seeds for x in col]))
            )
            
            score_new = np.trace(
                np.transpose(A) @ B[np.ix_(perm_inds_new, perm_inds_new)]
            )  # computing objective function value

            if obj_func_scalar * score_new < obj_func_scalar * score:  # minimizing
                score = score_new

                perm_inds = np.array([0] * n)
                perm_inds[p_A] = p_B[perm_inds_new]
                perm_inds = perm_inds.astype(int)

      
        p_A_unshuffle = np.array(range(n))
        p_A_unshuffle[p_A] = np.array(range(n))
        A = A[np.ix_(p_A_unshuffle, p_A_unshuffle)]
        p_B_unshuffle = np.array(range(n))
        p_B_unshuffle[p_B] = np.array(range(n))
        B = B[np.ix_(p_B_unshuffle, p_B_unshuffle)]
        score = np.trace(np.transpose(A) @ B[np.ix_(perm_inds, perm_inds)])
        
        self.perm_inds_ = perm_inds  # permutation indices
        self.score_ = score  # objective function value
        return self

    def fit_predict(self, A, B):
        """
        Fits the model with two assigned adjacency matrices, returning optimal
        permutation indices

        Parameters
        ---------
        A : 2d-array, square, positive
            A square, positive adjacency matrix

        B : 2d-array, square, positive
            A square, positive adjacency matrix

        Returns
        -------

        perm_inds_ : 1-d array, some shuffling of [0, n_vert)
            The optimal permutation indices to minimize the objective function
        """
        self.fit(A, B)
        return self.perm_inds_

In [16]:
print(np.concatenate([[],np.arange(5)], axis=None).astype(int))
with open("qapdata/chr12c.dat") as f:
    f = [int(elem) for elem in f.read().split()]
# adjusting
f = np.array(f[1:])
n = int(math.sqrt(len(f) / 2))
f = f.reshape(2 * n, n)
A = f[:n, :]
B = f[n:, :]
pi = np.array([7,5,1,3,10,4,8,6,9,11,2,12])-[1]*n

W1 = np.sort(random.sample(list(range(n)),3))
#W1 = np.array([3, 5, 9])
W2 = [pi[z] for z in W1]
faq = SGM(n_init=100,init_method="rand",shuffle_input=True)
#faq = SGM(shuffle_input=True)
#faq = faq.fit(A,B,W1,W2)
faq = faq.fit(A,B)

print(np.concatenate([[],np.arange(5)], axis=None))
print(W1)
print(W2)
print(pi)
print(faq.perm_inds_)
print(faq.score_)
print(np.trace(np.transpose(A) @ B[np.ix_(faq.perm_inds_, faq.perm_inds_)]))

[0 1 2 3 4]
[0. 1. 2. 3. 4.]
[0 4 7]
[6, 9, 5]
[ 6  4  0  2  9  3  7  5  8 10  1 11]
[ 5 10  3  7  0  9  1  6 11  4  8  2]
11188
11188


In [6]:
with open("qapdata/chr12c.dat") as f:
    f = [int(elem) for elem in f.read().split()]
# adjusting
f = np.array(f[1:])
n = int(math.sqrt(len(f) / 2))
f = f.reshape(2 * n, n)
A = f[:n, :]
B = f[n:, :]
pi = np.array([7,5,1,3,10,4,8,6,9,11,2,12])-[1]*n

for i in range(20):
    ofv = np.zeros(20)
    W1 = np.sort(random.sample(list(range(n)),3))
    #W1 = np.array([3, 5, 9])
    W2 = [pi[z] for z in W1]
    faq = SGM(n_init=100,init_method="rand",shuffle_input=True)
    #faq = SGM1(shuffle_input=True)
    faq = faq.fit(A,B,W1,W2)
    ofv[i] = faq.score_
    
print(max(ofv))

KeyboardInterrupt: 

In [5]:
with open("qapdata/chr15a.dat") as f:
    f = [int(elem) for elem in f.read().split()]
# adjusting
f = np.array(f[1:])
n = int(math.sqrt(len(f) / 2))
f = f.reshape(2 * n, n)
A = f[:n, :]
B = f[n:, :]
pi = np.array([5,10,8,13,12,11,14,2,4,6,7,15,3,1,9])-[1]*n

W1 = np.sort(random.sample(list(range(n)),3))
#W1 = np.array([3, 5, 9])
W2 = [pi[z] for z in W1]
faq = SGM(n_init=100,init_method="rand",shuffle_input=True)
#faq = SGM1(shuffle_input=True)
faq = faq.fit(A,B,W1,W2)

print(W1)
print(W2)
print(pi)
print(faq.perm_inds_)
print(faq.score_)
print(np.trace(np.transpose(A) @ B[np.ix_(faq.perm_inds_, faq.perm_inds_)]))

[ 2  7 10]
[7, 1, 6]
[ 4  9  7 12 11 10 13  1  3  5  6 14  2  0  8]
[ 4  9  7 12 11 10 13  1  3  5  6 14  2  0  8]
9896
9896


In [81]:
print (ofv)

[    0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0. 11156.]


In [11]:
W1 = []
W1 = column_or_1d(W1)
W1_c = np.array([x for x in range(n) if x not in W1])
p_A = np.concatenate(
            [W1, W1_c], axis=None
        ).astype(int)

In [15]:
print(p_A)
print(W1.shape[0])

[ 0  1  2  3  4  5  6  7  8  9 10 11]
0


In [30]:
A = A[np.ix_(p_A, p_A)]

In [31]:
print(A)

[[ 0 90 10  0  0  0  0  0  0  0  0  0]
 [90  0  0 23  0  0  0  0  0  0  0  0]
 [10  0  0  0 43  0  0  0  0  0  0  0]
 [ 0 23  0  0  0 88  0  0  0  0  0  0]
 [ 0  0 43  0  0  0 26  0  0  0  0  0]
 [ 0  0  0 88  0  0  0 16  0  0  0  0]
 [ 0  0  0  0 26  0  0  0  1  0  0  0]
 [ 0  0  0  0  0 16  0  0  0 96  0  0]
 [ 0  0  0  0  0  0  1  0  0  0 29  0]
 [ 0  0  0  0  0  0  0 96  0  0  0 37]
 [ 0  0  0  0  0  0  0  0 29  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 37  0  0]]


In [17]:
A11 = A[:0, :0]
print(A11)

[]


In [4]:
import networkx as nx


In [80]:
rand = nx.Graph()
boo = False
while boo == False:
    rand = nx.random_regular_graph(2,5)
    boo = nx.is_connected(rand)
rand = nx.convert_matrix.to_numpy_matrix(rand)
print(rand)
print(sum(rand))
print(sum(rand.transpose()))


[[0. 1. 1. 0. 0.]
 [1. 0. 0. 0. 1.]
 [1. 0. 0. 1. 0.]
 [0. 0. 1. 0. 1.]
 [0. 1. 0. 1. 0.]]
[[2. 2. 2. 2. 2.]]
[[2. 2. 2. 2. 2.]]


[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.

In [18]:
A = [[0,8,8,6,7],[8,0,12,10,11],[8,12,0,5,9],[6,10,5,0,15],[7,11,9,15,0]]
print(np.array(A))

[[ 0  8  8  6  7]
 [ 8  0 12 10 11]
 [ 8 12  0  5  9]
 [ 6 10  5  0 15]
 [ 7 11  9 15  0]]


In [50]:
A = [[0,2,100,12,5],[2,0,4,8,100],[100,4,0,3,3],[12,8,3,0,10],[5,100,3,10,0]]
print(np.array(A))

[[  0   2 100  12   5]
 [  2   0   4   8 100]
 [100   4   0   3   3]
 [ 12   8   3   0  10]
 [  5 100   3  10   0]]


In [51]:
faq = SGM(5,"rand")
faq = faq.fit(A,rand)
print(faq.score_)
print(faq.perm_inds_)
print(rand[np.ix_(faq.perm_inds_, faq.perm_inds_)])

42.0
[3 4 0 2 1]
[[0. 1. 0. 0. 1.]
 [1. 0. 0. 1. 0.]
 [0. 0. 0. 1. 1.]
 [0. 1. 1. 0. 0.]
 [1. 0. 1. 0. 0.]]


# Traveling Salesman Problem


In [5]:
dist = [
        [0, 2451, 713, 1018, 1631, 1374, 2408, 213, 2571, 875, 1420, 2145, 1972],
        [2451, 0, 1745, 1524, 831, 1240, 959, 2596, 403, 1589, 1374, 357, 579],
        [713, 1745, 0, 355, 920, 803, 1737, 851, 1858, 262, 940, 1453, 1260],
        [1018, 1524, 355, 0, 700, 862, 1395, 1123, 1584, 466, 1056, 1280, 987],
        [1631, 831, 920, 700, 0, 663, 1021, 1769, 949, 796, 879, 586, 371],
        [1374, 1240, 803, 862, 663, 0, 1681, 1551, 1765, 547, 225, 887, 999],
        [2408, 959, 1737, 1395, 1021, 1681, 0, 2493, 678, 1724, 1891, 1114, 701],
        [213, 2596, 851, 1123, 1769, 1551, 2493, 0, 2699, 1038, 1605, 2300, 2099],
        [2571, 403, 1858, 1584, 949, 1765, 678, 2699, 0, 1744, 1645, 653, 600],
        [875, 1589, 262, 466, 796, 547, 1724, 1038, 1744, 0, 679, 1272, 1162],
        [1420, 1374, 940, 1056, 879, 225, 1891, 1605, 1645, 679, 0, 1017, 1200],
        [2145, 357, 1453, 1280, 586, 887, 1114, 2300, 653, 1272, 1017, 0, 504],
        [1972, 579, 1260, 987, 371, 999, 701, 2099, 600, 1162, 1200, 504, 0],
    ]
dist = np.array(dist)
n = dist.shape[0]
rand = nx.Graph()
boo = False
while boo == False:
    rand = nx.random_regular_graph(2,n)
    boo = nx.is_connected(rand)
rand = nx.convert_matrix.to_numpy_matrix(rand)

In [11]:
faq = SGM(100,"rand")
faq = faq.fit(dist,rand)
print(faq.score_ / 2)
#print(faq.perm_inds_)
print(rand[np.ix_(faq.perm_inds_, faq.perm_inds_)])

7293.0
[[0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1.]
 [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0.]]
